In [1]:
import pandas as pd 
import geopandas as gpd
import gpflow
import numpy as np
import tensorflow as tf
from glob import glob

In [2]:
states = ["ILLINOIS", "IOWA", "MICHIGAN", "MINNESOTA", "OHIO", "INDIANA", "WISCONSON"]

In [3]:
df_yield = pd.read_csv("../../data/crops/soybean_yield_2015_2017.csv")
df_yield["key"] = df_yield["County"] + "-" + df_yield["State"]
counties_states = pd.read_csv("../../data/crops/counties-states.csv")
df_yield = df_yield[df_yield["key"].isin(counties_states["key"])]
# drop duplicated yields
df_yield = df_yield.drop_duplicates(["County", "State", "Year"])
df_modis = pd.read_csv("../../data/crops/processed_covariates/MOD13Q1-1000-all_data.csv")
df_gridmet = pd.read_csv("../../data/crops/processed_covariates/GRIDMET-all_data.csv")

In [4]:
df_yield.rename(columns={"Year": "year"}, inplace=True)

In [5]:
dates = ['04-07', '04-23', '05-09', '05-25',
       '06-10', '06-26', '07-12', '07-28',
       '08-13', '08-29', '09-14', '09-30',
       '10-16']

In [6]:
modis_features = ["year", "County", "State", "longitude", "latitude"] + [f"EVI_{date}" for date in dates]
df_modis = df_modis[modis_features]

In [7]:
df_modis = df_modis.groupby(["year","County", "State"]).mean().reset_index()

In [8]:
col_latlon = ["longitude", "latitude"]
col_modis = [f"EVI_{date}" for date in dates]
col_gridmet = [f"pr_{date}" for date in dates] + [f"tmmx_{date}" for date in dates]

In [9]:
gridmet_features = ["year", "County", "State"] + col_gridmet
df_gridmet = df_gridmet[gridmet_features]
df_gridmet = df_gridmet.groupby(["year", "County", "State"]).mean().reset_index()

In [10]:
df_features = df_modis.merge(df_gridmet, on=["County", "State", "year"])

In [11]:
df_all = df_yield.merge(df_features, on=["County", "State", "year"])

In [12]:
# df_all.to_csv("../../data/crops/centroid_yield_features_data.csv")
df_all.to_csv("../../data/crops/2015-2017centroid_yield_features_data.csv")

In [13]:
keys = set(df_all[df_all["year"]==2015].key).intersection(set(df_all[df_all["year"]==2017].key))
keys = list(keys)
keys.sort()

In [14]:
df_all = df_all[df_all["key"].isin(keys)]

In [16]:
col_index_space = [0, 1]
col_index_modis = [2, 3, 4, 5, 6, 7, 8, 9,10, 11, 12,13,14]
col_index_pr = [15+i for i in range(13)]
col_index_tmmx = [28+i for i in range(13)]

latlon_cols = ["longitude", "latitude"]
modis_cols = [f"EVI_{date}" for date in dates]
pr_cols = [f"pr_{date}" for date in dates] 
tmmx_cols = [f"tmmx_{date}" for date in dates] 
all_features = latlon_cols + modis_cols + pr_cols + tmmx_cols

assert df_all.shape[0] == 768


## Experimental Setup

In [18]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import gpflow 
from scipy.stats import sem
import json
import random 
import time 
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

seed = 0

X = df_all.loc[:,all_features].values
# X = df_all.iloc[:,6:34].values
y = df_all.Value.values[:, None]
kf = KFold(n_splits=5, random_state=seed, shuffle=True)
y = np.log(y)

## Centroid GP

In [19]:
RMSE = []
MAPE  = []
LL  = []
training_time = []

for fold, (train_index, test_index) in tqdm(enumerate(kf.split(keys))):
    # randomly pick counties
    county_keys = [keys[key] for key in train_index]
    train_index = (df_all["year"]==2015) & (df_all["key"].isin(county_keys))
    test_index = (df_all["year"]==2017) & (df_all["key"].isin(county_keys))
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    scaler_x = StandardScaler().fit(X_train)
    scaler_y = StandardScaler().fit(y_train)
    X_train, y_train = scaler_x.transform(X_train), scaler_y.transform(y_train)
    X_test = scaler_x.transform(X_test)

    # fit and train GP regression model
    k_space = gpflow.kernels.Matern32(active_dims=col_index_space)
    k_modis = gpflow.kernels.RBF(active_dims=col_index_modis)
    k_pr = gpflow.kernels.RBF(active_dims=col_index_pr)
    k_tmmx = gpflow.kernels.RBF(active_dims=col_index_tmmx)
    k = k_space + k_modis + k_pr + k_tmmx
    m = gpflow.models.GPR(data=(X_train, y_train), kernel=k, mean_function=None)
    t0 = time.time()
    opt = gpflow.optimizers.Scipy()
    opt_logs = opt.minimize(m.training_loss, m.trainable_variables, options=dict(maxiter=500))
    t1 = time.time()

    pred, var = m.predict_y(X_test)
    pred = scaler_y.inverse_transform(pred)
    # pred already inversely transformed, therefore only need to multiple the correction by scale_
    lower = np.reshape((1.96 * np.sqrt(var[:,0]))*scaler_y.scale_, y_test.shape)
    upper = np.reshape((1.96 * np.sqrt(var[:,0]))*scaler_y.scale_, y_test.shape)
    errors = np.concatenate((lower, upper), axis=1)
    errors = errors.T
    
    # compute metrics 
    loglik = np.mean(m.predict_log_density((X_test, scaler_y.transform(y_test))))
    RMSE.append(np.sqrt(np.mean((pred - y_test)**2)))
    MAPE.append(np.mean(np.abs(( pred - y_test) / y_test)))
    LL.append(loglik)

    # plot predictions
    plt.figure(figsize=(8,8))
    plt.scatter(pred, y_test, color="red")
    plt.plot(np.linspace(-100,100,201), np.linspace(-100,100,201), color="black")

    plt.errorbar(#
        pred[:,0],
        y_test[:,0],
        xerr=errors,
        fmt="o",
        ls="none",
        capsize=5,
        markersize=4,
        color="blue",
        alpha=0.2
        )
    plt.xlim((y_test.min()-0.5, y_test.max()+0.5))
    plt.ylim((y_test.min()-0.5, y_test.max()+0.5))
    plt.xlabel("Predictions")#
    plt.ylabel("Ground Truth")
    plt.savefig(f"../../results/usa_crops/centroidGP_fold{fold}.png")
    plt.savefig(f"../../results/usa_crops/centroidGP_fold{fold}.pdf")
    plt.savefig(f"../../results/usa_crops/centroidGP_fold{fold}.svg")
    plt.close()

print(f"MAPE", MAPE)
print(f"RMSE", RMSE)
print(f"LL", LL)
print(f"Training Time", training_time)

0it [00:00, ?it/s]2022-01-14 15:25:48.277568: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-01-14 15:25:48.277610: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: nvidia4
2022-01-14 15:25:48.277615: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: nvidia4
2022-01-14 15:25:48.277746: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 460.27.4
2022-01-14 15:25:48.277765: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 460.27.4
2022-01-14 15:25:48.277769: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 460.27.4
2022-01-14 15:25:48.278181: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU 

MAPE [0.018791189083751776, 0.020581927259052633, 0.02066895792313438, 0.020093082676521812, 0.020776552531332395]
RMSE [0.09548382258497803, 0.10308572558106202, 0.10853600556165986, 0.10308398535646551, 0.10317844268578949]
LL [-0.7412401353454348, -0.7476352550037826, -0.8043362508449573, -0.7227097675091686, -0.8004552292144533]
Training Time []


In [20]:
m

name,class,transform,prior,trainable,shape,dtype,value
GPR.kernel.kernels[0].variance,Parameter,Softplus,,True,(),float64,0.194403
GPR.kernel.kernels[0].lengthscales,Parameter,Softplus,,True,(),float64,0.416133
GPR.kernel.kernels[1].variance,Parameter,Softplus,,True,(),float64,1.82891
GPR.kernel.kernels[1].lengthscales,Parameter,Softplus,,True,(),float64,7.75928
GPR.kernel.kernels[2].variance,Parameter,Softplus,,True,(),float64,0.473155
GPR.kernel.kernels[2].lengthscales,Parameter,Softplus,,True,(),float64,22.3012
GPR.kernel.kernels[3].variance,Parameter,Softplus,,True,(),float64,0.0407635
GPR.kernel.kernels[3].lengthscales,Parameter,Softplus,,True,(),float64,2.37322
GPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.0488889


In [20]:
json_file = json.dumps({"CV-RMSE": sum(RMSE) / 5, "CV-MAPE": sum(MAPE) / 5, 
                       "CV-sd-RMSE": sem(RMSE), "CV-sd-MAPE": sem(MAPE), "CV-LL": sum(LL) / 5, "CV-sd-LL": sem(LL),
                        "Training Time": sum(training_time)/5, "Training Time se": sem(training_time)}
                       )
f = open(f"../../results/usa_crops/centroidGP.json", "w")
f.write(json_file)
f.close()

/home/hbz15/MVBAgg/.env/lib/python3.8/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/hbz15/MVBAgg/.env/lib/python3.8/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/hbz15/MVBAgg/.env/lib/python3.8/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [21]:
# alpha = K_ZZ^{-1} @ m
list_columns = [latlon_cols, modis_cols, pr_cols, tmmx_cols]
alpha = m.posterior().alpha 
Z = X_train
sobol = {}
all_features = []
for l in [latlon_cols, modis_cols, pr_cols, tmmx_cols]:
    all_features += l
df_all_scaled = df_all.copy()
df_all_scaled.loc[:, all_features] = scaler_x.transform(df_all_scaled.loc[:, all_features].values)

Kmn = m.kernel(df_all_scaled[all_features].values, Z)
alpha_j = Kmn @ alpha
variance_full = np.mean(alpha_j**2) - np.mean(alpha_j)**2 + m.likelihood.variance.numpy()

alpha_list = []
for i, cols in enumerate(list_columns):
    Z_tmp = tf.gather(Z, indices=m.kernel.kernels[i].active_dims, axis=1)
    Kmn = m.kernel.kernels[i].K(X[cols].values, Z_tmp)
    alpha_j = Kmn @ alpha
    alpha_list.append(alpha_j)

# compute the 1st order Sobols
for i in range(len(list_columns)):
    sobol[f"{i}"] = (
        np.mean(alpha_list[i] ** 2) - np.mean(alpha_list[i]) ** 2
    ) / variance_full

# compute the 2nd order Sobols
for i in range(len(list_columns)):
    for j in range(i+1, len(list_columns)):
        sobol[f"{i}-{j}"] = (
            np.mean(
                2
                * (alpha_list[i] - np.mean(alpha_list[i]))
                * (alpha_list[j] - np.mean(alpha_list[j]))
            )
            / variance_full
        )


In [22]:
sobol

[0.01388125404471421,
 0.7790494374804148,
 0.00012744308599918497,
 0.0008302887195146124]

## Random Forest

In [23]:
from sklearn.ensemble import RandomForestRegressor

RMSE = []
MAPE  = []
training_time = []

for fold, (train_index, test_index) in tqdm(enumerate(kf.split(keys))):
    # randomly pick counties
    county_keys = [keys[key] for key in train_index]
    train_index = (df_all["year"]==2015) & (df_all["key"].isin(county_keys))
    test_index = (df_all["year"]==2017) & (df_all["key"].isin(county_keys))
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    scaler_x = StandardScaler().fit(X_train)
    scaler_y = StandardScaler().fit(y_train)
    X_train, y_train = scaler_x.transform(X_train), scaler_y.transform(y_train)
    X_test = scaler_x.transform(X_test)

    # fit and train GP regression model
    m = RandomForestRegressor(max_depth=2, random_state=0)   

    t0 = time.time()
    m.fit(X_train, y_train.ravel())    
    t1 = time.time()

    pred = m.predict(X_test)
    pred = scaler_y.inverse_transform(pred[:, None])
    
    # compute metrics 
    RMSE.append(np.sqrt(np.mean((pred - y_test)**2)))
    MAPE.append(np.mean(np.abs(( pred - y_test) / y_test)))
    training_time.append(t1-t0)

    # plot predictions
    plt.figure(figsize=(8,8))
    plt.scatter(pred, y_test, color="red")
    plt.plot(np.linspace(-100,100,201), np.linspace(-100,100,201), color="black")
    plt.xlim((y_test.min()-0.5, y_test.max()+0.5))
    plt.ylim((y_test.min()-0.5, y_test.max()+0.5))
    plt.xlabel("Predictions")
    plt.ylabel("Ground Truth")
    plt.savefig(f"../../results/usa_crops/centroidRF_fold{fold}.png")
    plt.savefig(f"../../results/usa_crops/centroidRF_fold{fold}.pdf")
    plt.close()

print(f"MAPE", MAPE)
print(f"RMSE", RMSE)
print(f"Training Time", training_time)

json_file = json.dumps({"CV-RMSE": sum(RMSE) / 5, "CV-MAPE": sum(MAPE) / 5, 
                       "CV-sd-RMSE": sem(RMSE), "CV-sd-MAPE": sem(MAPE), 
                        "Training Time": sum(training_time)/5, "Training Time se": sem(training_time)}

                       )
f = open(f"../../results/usa_crops/centroidRF.json", "w")
f.write(json_file)
f.close()

5it [00:01,  3.28it/s]

MAPE [0.020278537632900263, 0.021368765169560877, 0.020510880243303845, 0.022660574395256823, 0.02192326580755986]
RMSE [0.10153536445220938, 0.10632312651863317, 0.1043423781526632, 0.1142315398900619, 0.1091623436432785]
Training Time [0.16010594367980957, 0.15932917594909668, 0.15838003158569336, 0.15716767311096191, 0.15930533409118652]
